In [1]:
pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.


In [1]:
import mysql.connector
import os
import mysql.connector
import matplotlib.pyplot as plt
from datetime import datetime


def getConnection():
    return mysql.connector.connect(host="localhost", user="root", password="", database="railway")

class User:
    def __init__(self, user_id, name, age, contact):
        self.user_id = user_id
        self.name = name
        self.age = age
        self.contact = contact

    def __str__(self):
        return f"ID: {self.user_id}, Name: {self.name}, Age: {self.age}, Contact: {self.contact}"



In [2]:
def new_user():
    connection = None
    cursor = None
    try:
        name = input("Enter Your Name (max 50 chars): ")
        if len(name) > 50:
            print("❌ Use less than 50 characters.")
            return

        age = input("Enter Your Age: ")
        if not age.isdigit():
            print("❌ Invalid Age. Enter a numeric value.")
            return

        contactNo = input("Enter Your Contact Number: ")
        if len(contactNo) != 10 or not contactNo.isdigit():
            print("❌ Invalid Contact Number. Enter exactly 10 digits.")
            return

        connection = getConnection()
        cursor = connection.cursor()
        cursor.execute("insert into user (name, age, contact) value (%s, %s, %s)", (name, age, contactNo))
        connection.commit()

        cursor.execute("select LAST_INSERT_ID()")
        user_id = cursor.fetchone()[0]

        print(f"✅ User registered successfully! User ID: {user_id}")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [3]:
def fetch_user_by_id():
    connection = None
    cursor = None
    try:
        user_id = input("Enter User ID to fetch: ")
        if not user_id.isdigit():
            print("❌ Invalid ID. Please enter a numeric ID.")
            return

        connection = getConnection()
        cursor = connection.cursor()
        cursor.execute("select id, name, age, contact from user where id = %s", (user_id,))
        record = cursor.fetchone()

        if record:
            print("\n✅ User Found:\n", User(*record))
        else:
            print("❌ No user found with the given ID.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [4]:
def fetch_user_by_name_contact():
    connection = None
    cursor = None
    try:
        name = input("Enter Name to fetch: ")
        contactNo = input("Enter Contact Number: ")

        if len(contactNo) != 10 or not contactNo.isdigit():
            print("❌ Invalid Contact Number. Enter exactly 10 digits.")
            return

        connection = getConnection()
        cursor = connection.cursor()
        cursor.execute("select id, name, age, contact from user where name = %s AND contact = %s", (name, contactNo))
        records = cursor.fetchall()

        if records:
            print("\n✅ Users Found:")
            for record in records:
                print(User(*record))
        else:
            print("❌ No user found with the given name and contact number.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [5]:
def delete_user_by_id():
    connection = None
    cursor = None
    try:
        user_id = input("Enter User ID to delete: ")
        if not user_id.isdigit():
            print("❌ Invalid ID. Please enter a numeric ID.")
            return

        connection = getConnection()
        cursor = connection.cursor()
        cursor.execute("select name from user where id = %s", (user_id,))
        record = cursor.fetchone()

        if not record:
            print("❌ No user found with the given ID.")
            return
        
        user_name = record[0]
        confirm = input(f"⚠ Are you sure you want to delete User '{user_name}' (ID: {user_id})? (yes/no): ").strip().lower()
        if confirm != "yes":
            print("❌ Deletion cancelled.")
            return

        cursor.execute("delete from user WHERE id = %s", (user_id,))
        connection.commit()
        print(f"✅ User '{user_name}' (ID: {user_id}) deleted successfully.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [6]:
def passenger_control():
    while True:
        print("\n====== 🚆 Passenger Control Panel 🚆 ======")
        print("1️⃣ Register a New User")
        print("2️⃣ Fetch User by ID")
        print("3️⃣ Fetch User by Name & Contact")
        print("4️⃣ Delete User by ID")
        print("5️⃣ Back to Home")
        print("==============================================")
        choice = input("Enter your choice (1-5): ")

        if choice == "1":
            new_user()
        elif choice == "2":
            fetch_user_by_id()
        elif choice == "3":
            fetch_user_by_name_contact()
        elif choice == "4":
            delete_user_by_id()
        elif choice == "5":
            break
        else:
            print("❌ Invalid choice. Please enter a number between 1 and 5.")


In [7]:
def add_train():
    connection = None
    cursor = None
    try:
        train_no = input("Enter Train Number: ")
        name = input("Enter Train Name: ")
        date = input("Enter Train Date (DD-MM-YYYY): ")
        from_station = input("Enter From Station: ")
        to_station = input("Enter To Station: ")
        departure = input("Enter Departure Time (HH:MM): ")
        arrival = input("Enter Arrival Time (HH:MM): ")
        available_seat = input("Enter Available Seats: ")
        basefare = input("Enter Base Fare: ")

        if not train_no.isdigit() or not available_seat.isdigit() or not basefare.isdigit():
            print("❌ Train Number, Available Seats, and Base Fare should be numeric values.")
            return

        connection = getConnection()
        cursor = connection.cursor()
        query = "insert into trains value (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(query, (train_no, name, date, from_station, to_station, departure, arrival, available_seat, basefare))
        connection.commit()

        print(f"✅ Train {name} (No: {train_no}) added successfully!")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [8]:
def print_train_details(record):
    print("\n🚆 Train Details:")
    print("════════════════════════════════════")
    print(f"📌 Train Number  : {record[0]}")
    print(f"📛 Name          : {record[1]}")
    print(f"📅 Date          : {record[2]}")
    print(f"🚉 From Station  : {record[3]}")
    print(f"🏁 To Station    : {record[4]}")
    print(f"⏰ Departure     : {record[5]}")
    print(f"⏳ Arrival        : {record[6]}")
    print(f"🎟 Available Seats: {record[7]}")
    print(f"💰 Base Fare      : ₹{record[8]}")
    print("════════════════════════════════════\n")


In [9]:
def find_train_by_number():
    connection = None
    cursor = None
    try:
        train_no = input("Enter Train Number to Search: ")
        if not train_no.isdigit():
            print("❌ Invalid Train Number. Enter a numeric value.")
            return

        connection = getConnection()
        cursor = connection.cursor()
        query = "select * from trains where Train_No = %s"
        cursor.execute(query, (train_no,))
        record = cursor.fetchone()

        if record:
            print_train_details(record)
        else:
            print("❌ No train found with the given Train Number.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [10]:
def find_train_by_route():
    connection = None
    cursor = None
    try:
        from_station = input("Enter From Station: ")
        to_station = input("Enter To Station: ")

        connection = getConnection()
        cursor = connection.cursor()
        query = "select * from trains where From_station = %s AND To_station = %s"
        cursor.execute(query, (from_station, to_station))
        records = cursor.fetchall()

        if records:
            print("\n✅ Trains Available:")
            for record in records:
                print_train_details(record)
        else:
            print("❌ No trains found for the given route.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [11]:

def get_todays_trains():
    connection = None
    cursor = None
    try:
        today_date = datetime.today().strftime('%d-%m-%Y')

        connection = getConnection()
        cursor = connection.cursor()
        query = "select * from trains where Date = %s"
        cursor.execute(query, (today_date,))
        records = cursor.fetchall()

        if records:
            print("\n✅ Trains Running Today:")
            for record in records:
                print_train_details(record)
        else:
            print("❌ No trains found for today.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [12]:
def delete_train():
    connection = None
    cursor = None
    try:
        train_no = input("Enter Train Number to Delete: ")
        if not train_no.isdigit():
            print("❌ Invalid Train Number. Enter a numeric value.")
            return

        connection = getConnection()
        cursor = connection.cursor()
        cursor.execute("select Name from trains where Train_No = %s", (train_no,))
        record = cursor.fetchone()

        if not record:
            print("❌ No train found with the given Train Number.")
            return

        train_name = record[0]
        confirm = input(f"⚠ Are you sure you want to delete Train '{train_name}' (No: {train_no})? (yes/no): ").strip().lower()
        if confirm != "yes":
            print("❌ Deletion cancelled.")
            return

        cursor.execute("delete from trains where Train_No = %s", (train_no,))
        connection.commit()
        print(f"✅ Train '{train_name}' (No: {train_no}) deleted successfully.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [13]:
def train_control():
    while True:
        print("\n====== 🚆 Train Control Panel 🚆 ======")
        print("1️⃣ Add Train")
        print("2️⃣ Find Train by Number")
        print("3️⃣ Find Train by From & To")
        print("4️⃣ Get Today's Trains")
        print("5️⃣ Delete Train")
        print("6️⃣ Back to Home")
        print("==========================================")

        choice = input("Enter your choice (1-6): ")

        if choice == "1":
            add_train()
        elif choice == "2":
            find_train_by_number()
        elif choice == "3":
            find_train_by_route()
        elif choice == "4":
            get_todays_trains()
        elif choice == "5":
            delete_train()
        elif choice == "6":
            break
        else:
            print("❌ Invalid choice. Please enter a number between 1 and 6.")


In [14]:
def book_train():
    try:
        connection = getConnection()
        cursor = connection.cursor()

        user_id = input("Enter User ID: ").strip()
        train_no = input("Enter Train Number: ").strip()
        seat_count = int(input("Enter Number of Seats: ").strip())

        # Check if User ID is valid
        cursor.execute("select id from user where id = %s", (user_id,))
        if not cursor.fetchone():
            print("❌ Invalid User ID.")
            return

        # Check if Train Number is valid
        cursor.execute("select Available_seat, Basefare, Name, Date, From_Station, To_Station, Departure, Arrival from trains WHERE Train_No = %s", (train_no,))
        train = cursor.fetchone()
        if not train:
            print("❌ Invalid Train Number.")
            return

        available_seat, base_fare, train_name, train_date, from_station, to_station, departure, arrival = train

        # Check seat availability
        if seat_count > available_seat:
            print("❌ Not enough seats available.")
            return

        # minus seats from train availability
        cursor.execute("update trains set Available_seat = Available_seat - %s where Train_No = %s", (seat_count, train_no))
        connection.commit()

        # Payment 
        amount = seat_count * base_fare
        print(f"\nYou have to pay total of : {amount}")
        print("\nSelect Payment Method:")
        print("1️⃣ Cash")
        print("2️⃣ Card")
        print("3️⃣ UPI")
        print("4️⃣ Net Banking")
        payment_choice = input("Enter choice (1-4): ").strip()

        if payment_choice == "2":
            method_detail = input("Enter Card Number: ").strip()
            payment_method = "Card"
        elif payment_choice == "3":
            method_detail = input("Enter UPI ID: ").strip()
            payment_method = "UPI"
        elif payment_choice == "4":
            method_detail = input("Enter Net Banking Details: ").strip()
            payment_method = "Net Banking"
        else:
            method_detail = "Cash Payment"
            payment_method = "Cash"

        # Insert Payment Data
        cursor.execute("insert into payments (UserId, amount, payment_method, method_detail) value (%s, %s, %s, %s)",
                       (user_id, amount, payment_method, method_detail))
        connection.commit()

        payment_id = cursor.lastrowid

        # Insert Booking Data
        cursor.execute("insert into bookings (SeatNo, PaymentId, UserId, TrainNo) value (%s, %s, %s, %s)",
                       (seat_count, payment_id, user_id, train_no))
        connection.commit()

        booking_id = cursor.lastrowid

        print(f"✅ Booking Successful! Payment ID: {payment_id}, Amount Paid: {amount}, Seats: {seat_count}")


        cursor.execute("select * from user where id = %s", (user_id,))
        user = cursor.fetchone()
        user_name = user[1]
        user_contact = user[3]

        

        
        # Generate Ticket File
        ticket_filename = f"Tickets/ticket_{booking_id}.txt"
        with open(ticket_filename, "w") as ticket_file:
            ticket_file.write("\n----------------------------------\n")
            ticket_file.write(f"          TRAIN TICKET         \n")
            ticket_file.write("----------------------------------\n")
            ticket_file.write(f"Booking ID: {booking_id}\n")
            ticket_file.write(f"Passenger ID: {user_id}\n")
            ticket_file.write(f"Passenger Name: {user_name}\n")
            ticket_file.write(f"Passenger Contact: {user_contact}\n")
            ticket_file.write(f"Train No: {train_no}\n")
            ticket_file.write(f"Train Name: {train_name}\n")
            ticket_file.write(f"Date of travel: {train_date}\n")
            ticket_file.write(f"From: {from_station}  To: {to_station}\n")
            ticket_file.write(f"Departure: {departure}  Arrival: {arrival}\n")
            ticket_file.write(f"Seat No(s): {seat_count}\n")
            ticket_file.write(f"Payment Method: {payment_method}\n")
            ticket_file.write(f"Total Amount Paid: Rs. {amount}\n")
            ticket_file.write("----------------------------------\n")

        print(f"📄 Ticket saved as {ticket_filename}")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [15]:
def check_booking():
    booking_id = input("Enter Booking ID: ").strip()
    try:
        connection = getConnection()
        cursor = connection.cursor()

        cursor.execute("select * from bookings where BookingId = %s", (booking_id,))
        booking = cursor.fetchone()

        if not booking:
            print("❌ Booking not found.")
            return

        ticket_filename = open(f"Tickets/ticket_{booking[0]}.txt", 'r')
        print(ticket_filename.read())

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [16]:
def view_recent_bookings():
    try:
        connection = getConnection()
        cursor = connection.cursor()

        cursor.execute("select * from bookings order by BookingId desc limit 10")
        bookings = cursor.fetchall()

        if not bookings:
            print("No recent bookings found.")
            return

        print("\nRecent Bookings:")
        print("----------------------")
        for booking in bookings:
            print(f"Booking ID: {booking[0]}")
            print(f"Seat No: {booking[1]}")
            print(f"Payment ID: {booking[2]}")
            print(f"User ID: {booking[3]}")
            print(f"Train No: {booking[4]}")
            print("----------------------")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [17]:
def cancel_booking():
    booking_id = input("Enter Booking ID to cancel: ").strip()
    try:
        connection = getConnection()
        cursor = connection.cursor()

        cursor.execute("select SeatNo, TrainNo from bookings where BookingId = %s", (booking_id,))
        booking = cursor.fetchone()

        if not booking:
            print("❌ Booking not found.")
            return

        seat_no,train_no = booking

        confirm = input(f"⚠ Are you sure you want to cancel booking of '{train_no}'? (yes/no): ").strip().lower()
        if confirm != "yes":
            print("❌ Deletion cancelled.")
            return
        
        cursor.execute("update trains set Available_seat = Available_seat + %s where Train_No = %s", (seat_no, train_no))
        cursor.execute("delete from bookings where BookingId = %s", (booking_id,))
        connection.commit()

        print("✅ Booking cancelled successfully.")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()


In [18]:
def booking_control():
    while True:
        print("\n====== 📄 Booking Control 📄 ======")
        print("1️⃣ Book Train")
        print("2️⃣ Check Booking")
        print("3️⃣ View Recent Booking")
        print("4️⃣ Cancel Booking")
        print("5️⃣ Back to Home")
        print("=====================================")
        choice = input("Enter choice (1-5): ").strip()

        if choice == "1":
            book_train()
        elif choice == "2":
            check_booking()
        elif choice == "3":
            view_recent_bookings()
        elif choice == "4":
            cancel_booking()
        elif choice == "5":
            break
        else:
            print("❌ Invalid choice.")


In [19]:

def bestseller_trains_report():
    try:
        connection = getConnection()
        cursor = connection.cursor()

        cursor.execute("""
            select t.Name, SUM(p.amount) as total_sales from payments p
            join bookings b on p.Paymentid = b.PaymentId
            join trains t on b.TrainNo = t.Train_No
            group by t.Name
            order by total_sales desc
        """)
        
        data = cursor.fetchall()
        if not data:
            print("No sales data available.")
            return

        train_names = [train[0] for train in data]
        sales = [train[1] for train in data]

        
        plt.figure(figsize=(8, 6))
        plt.pie(sales, labels=train_names, autopct='%1.1f%%', startangle=90)
        plt.title("Bestseller Trains by Total Sales")

        
        report_dir = os.path.join(os.getcwd(), "Reports")
        os.makedirs(report_dir, exist_ok=True)
        file_path = os.path.join(report_dir, "bestseller_trains.png")
        plt.savefig(file_path)
        plt.show()

        print(f"✅ Bestseller Trains Report saved at: {file_path}")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        cursor.close()
        connection.close()


In [20]:

def most_booked_trains_report():
    try:
        connection = getConnection()
        cursor = connection.cursor()

        
        cursor.execute("""
            select t.Name, Sum(b.SeatNo) as total_booked
            from bookings b
            join trains t ON b.TrainNo = t.Train_No
            group by t.Name
            order by total_booked desc
        """)
        
        data = cursor.fetchall()
        if not data:
            print("No booking data available.")
            return

        train_names = [t[0] for t in data]
        booked_counts = [t[1] for t in data]

        # Plot Bar graph
        plt.figure(figsize=(10, 5))
        plt.barh(train_names, booked_counts, color='skyblue')
        plt.xlabel("Train Name")
        plt.ylabel("Seats Booked")
        plt.title("Most Booked Trains")
        plt.tight_layout()
    


        # Save Graphs in 'Reports' folder
        report_dir = os.path.join(os.getcwd(), "Reports")
        os.makedirs(report_dir, exist_ok=True)
        file_path = os.path.join(report_dir, "most_booked_trains.png")
        plt.savefig(file_path)
        plt.show()

        print(f"✅ Most Booked Trains Report saved at: {file_path}")

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
    finally:
        cursor.close()
        connection.close()



In [21]:
def get_reports():
    while True:
        print("\n====== 📊 Get Reports📊  ======")
        print("1️⃣ Bestseller Trains (By Total Sales - Pie Chart)")
        print("2️⃣ Most Booked Train (By Number of Seats - Bar graph)")
        print("3️⃣ Back to Home")
        print("==================================")

        choice = input("Enter choice (1-3): ")

        if choice == "1":
            bestseller_trains_report()
        elif choice == "2":
            most_booked_trains_report()
        elif choice == "3":
            break
        else:
            print("❌ Invalid choice. Please enter a valid option.")



In [22]:
def home():
    while True:
        print("\n====== 🚆 Train Booking System 🚆 ======")
        print("1️⃣ Passenger Control")
        print("2️⃣ Train Control")
        print("3️⃣ Booking Control")
        print("4️⃣ Get Reports")
        print("5️⃣ EXIT from system")
        print("="*41)
        choice = input("Enter your choice (1-5): ")

        if choice == "1":
            passenger_control()
        elif choice == "2":
            train_control()
        elif choice == "3":
            booking_control()
        elif choice == "4":
            get_reports()
        elif choice == "5":
            print("Exiting, Have a nice day! 👋")
            break
        else:
            print("❌ Invalid choice. Please enter a number between 1 and 5.")


In [1]:
home()

NameError: name 'home' is not defined